# Woa, Gwo, Pso, SMA evaluator– Combined Notebook

## 1. Common imports

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

## 2. Whale Optimization Algorithm implementation

In [ ]:
import numpy as np

class WOA_Optimizer:
    ''' fitness: the fitness function as an input
        seed: seed of random generator
        dim: the number of hyperparameters or equivalently the dim of the hyperparameter vector.
        Xmin: the lower bounds for the hyperparameters, should has length == dim
        Xmax: the upper bounds for the hyperparameters, should has length == dim
        populationNumber: the number of population
        max_iter: iteration number 
    '''
    def __init__(self, fitness, seed, dim, Xmin, Xmax, populationNumber, max_iter, scale):
        self.X_array=[]
        self.max_iter = max_iter

        self.dim = dim 
        self.Xmin = Xmin
        self.Xmax = Xmax
        self.Fitness = fitness
        self.rng = np.random.default_rng(seed)
        self.n  = populationNumber #poplulation number
        self.timeScale = scale
        ##these are internal paramters of the algorithm
        self.a = 2 #a is linearly decreased from 2 to 0 over the course
        self.r = 0 #random vector in [0,1]
        self.A = 0 #2 * a * r - a
        self.C = 0 #2 * r
        self.l = 0 #random number in [-1,1]
        self.p = 0 #random number in [0,1]
        self.b = 1 #const to define log spiral

        return
    
    #in each iteration, update the internal parameters
    def Update(self):
        self.a =  self.a -  2.0 /self.max_iter
        self.r = self.rng.uniform(0, 1)
        self.A = self.a * self.r * 2 - self.a
        self.C = self.r * 2.0
        self.l = self.rng.uniform(-1, 1)
        self.p = self.rng.uniform(0, 1)

        return

    #corresponding to the first equation in the algo
    def Equation1(self, X , X_target):
        D = np.abs(X_target * self.C - X)
        return D

    #corresponding to the second equation in the algo
    def Equation2(self,X, X_target):
        D_ = np.abs(X_target - X)
        X_output = D_ * np.exp(self.b * self.l) * np.cos(2* np.pi * self.l) + X_target
        return X_output

    #corresponding to the third equation in the algo
    def Equation3(self, X,X_rand):
        D = np.abs(X_rand * self.C - X)
        X_output = X_rand - D * self.A
        return X_output

    #in case of out of bound
    def Amend(self, X_array):
        for X in X_array:
            for j in range(self.dim):
                if X[j] < self.Xmin[j]:
                    X[j] = self.Xmin[j]
                elif X[j] > self.Xmax[j]:
                    X[j] = self.Xmax[j]
        return

    #generate a random candidate
    def RandomAgent(self):
        Xs = np.zeros(self.dim)
        for i in range(0, self.dim):
            Xs[i] = (self.Xmax[i] - self.Xmin[i]) * self.rng.uniform(0,1) + self.Xmin[i]
        return Xs

    #Initialize the whale population Xi(i = 1, 2, ..., n)
    def RandomPopulation(self):
        X_array = np.zeros((self.n, self.dim))
        for i in range(self.n):
            X_array[i] = self.RandomAgent()
        return X_array

    #Calculate the fitness of each search agent and find the best one
    def FindBestAgent(self, X_array ,X_target):
        fitness = 0
        if X_target is not None:
            fitness_target = self.Fitness(X_target, self.timeScale)
        else:
            fitness_target = self.Fitness(X_array[0], self.timeScale)
            X_target = X_array[0]
        for X in X_array:
            fitness = self.Fitness(X, self.timeScale)
            if fitness > fitness_target:
                #Update X* if there is a better solution
                fitness_target = fitness
                X_target = X
        X_target = X_target.copy()
        return X_target, fitness_target


    #main function, run the algorithm
    def Run(self):
        X_target = None
        #Initialize the whales population Xi(i = 1, 2, ..., n) 
        X_array = self.RandomPopulation()
        #Calculate the fitness of each search agent
        #X*=the best search agent
        X_target, fit = self.FindBestAgent(X_array, None)

        fits = []
        t = 0
        while t < self.max_iter:
            for i in range(len(X_array)):
                X = X_array[i]
                X1 = None
                self.Update()
                if self.p < 0.5:
                    if abs(self.A) < 1:
                        X1 = self.Equation1(X, X_target)
                    else:
                        X_rand = self.RandomAgent()
                        X1 = self.Equation3(X,X_rand)
                else:
                  X1 = self.Equation2(X,X_target)

                X_array[i] = X1

            #Check if any search agent goes beyond the search space and amend its position
            self.Amend(X_array)

            #Calculate the fitness of each search agent
            #Update X* if there is a better solution
            X_target, fit = self.FindBestAgent(X_array, X_target)
            # print(f"{t} current_best_fitness:", fit)
            fits.append(fit)

            t = t + 1

        return  X_target, fit, fits


## 3. Additional optimisation algorithms

### 3.1 Grey Wolf Optimiser (GWO) implementation

In [ ]:
import numpy as np

class GreyWolfOptimiser:
    def __init__(self, fitness_function, max_iter, num_wolves, dim, minx, maxx, seed=0, fitness_args=None):
        self.fitness = fitness_function
        self.fitness_args = fitness_args if fitness_args is not None else ()
        self.max_iter = max_iter
        self.num_wolves = num_wolves
        self.dim = dim # dimension
        self.minx = np.array(minx)
        self.maxx = np.array(maxx)
        self.rng = np.random.default_rng(seed)

        # Initial wolf positions
        self.positions = self.rng.uniform(minx, maxx, size=(num_wolves, dim))
        self.fitness_values = np.array([fitness_function(pos, *self.fitness_args) for pos in self.positions])
        
        # Initialise alpha, beta, and delta positions
        self.alpha_wolf = None
        self.beta_wolf = None
        self.delta_wolf = None
        self.alpha_score = 0
        self.beta_score = 0
        self.delta_score = 0

    def update_leader_positions(self):
        # Sort wolves from maximum fitness to minimum fitness
        sorted_indices = np.argsort(self.fitness_values)[::-1]
        
        # Update alpha, beta, and delta positions
        self.alpha_wolf = self.positions[sorted_indices[0]] # best wolf
        self.beta_wolf = self.positions[sorted_indices[1]] # second best wolf
        self.delta_wolf = self.positions[sorted_indices[2]] # third best wolf
        
        # Update their scores
        self.alpha_score = self.fitness_values[sorted_indices[0]] # best fitness value
        self.beta_score = self.fitness_values[sorted_indices[1]] # second best fitness value
        self.delta_score = self.fitness_values[sorted_indices[2]] # third best fitness value

    def update_wolf_positions(self, iteration):
        # a linearly decreases from 2 to 0
        a = 2 - iteration * (2 / self.max_iter)
        
        for i in range(self.num_wolves):
            # Update each wolf's position
            for j in range(self.dim):
                r1 = self.rng.random()
                r2 = self.rng.random()
                A1 = 2 * a * r1 - a
                C1 = 2 * r2
                
                # D_alpha = distance between alpha wolf and current wolf
                D_alpha = abs(C1 * self.alpha_wolf[j] - self.positions[i, j])
                X1 = self.alpha_wolf[j] - A1 * D_alpha
                
                r1 = self.rng.random()
                r2 = self.rng.random()
                A2 = 2 * a * r1 - a
                C2 = 2 * r2
                
                # D_beta = distance between beta wolf and current wolf
                D_beta = abs(C2 * self.beta_wolf[j] - self.positions[i, j])
                X2 = self.beta_wolf[j] - A2 * D_beta
                
                r1 = self.rng.random()
                r2 = self.rng.random()
                A3 = 2 * a * r1 - a
                C3 = 2 * r2
                
                # D_delta = distance between delta wolf and current wolf
                D_delta = abs(C3 * self.delta_wolf[j] - self.positions[i, j])
                X3 = self.delta_wolf[j] - A3 * D_delta
                
                # Update position
                self.positions[i, j] = (X1 + X2 + X3) / 3
                
                # Keep position within bounds
                self.positions[i, j] = np.clip(self.positions[i, j], self.minx[j], self.maxx[j])
            
            # Update the fitness value
            self.fitness_values[i] = self.fitness(self.positions[i], *self.fitness_args)

    def run(self):
        # Initialise leader (alpha, beta, delta) positions
        self.update_leader_positions()
        
        for iteration in range(self.max_iter):
            # Update wolf positions
            self.update_wolf_positions(iteration)
            
            # Update leader positions
            self.update_leader_positions()
        
        return self.alpha_wolf, self.alpha_score

# TESTING
# def fitness_function(x):
#     return sum(x**2)  # sum of squares

# gwo = GreyWolfOptimiser(
#     fitness_function=fitness_function,
#     max_iter=100,
#     num_wolves=30,
#     dim=5,
#     minx=-10,
#     maxx=10
# )   

# best_solution, best_fitness = gwo.run()
# print(f"Best solution: {best_solution}")
# print(f"Best fitness: {best_fitness}")

### 3.2 Particle Swarm Optimisation (PSO) implementation

In [ ]:
#particle swarm optimization

import numpy as np

class PSO:
  def __init__(self, fitness, seed, dim, Xmin, Xmax, populationNumber, max_iter, timeScale, w = 1.0 , sigma_p = 2.0, sigma_g = 2.0):
    self.populationNumber = populationNumber
    self.dim = dim
    self.Fitness = fitness
    self.Xmin = Xmin
    self.Xmax = Xmax
    self.max_iter = max_iter
    self.rng = np.random.default_rng(seed)
    self.w = w
    self.sigma_p = sigma_p
    self.sigma_g = sigma_g
    self.timeScale = timeScale
    return 
  
  def Run(self):
    Xarray = np.zeros((self.populationNumber, self.dim))
    Varray = np.zeros((self.populationNumber, self.dim))
    XbestArray = np.zeros((self.populationNumber, self.dim))
    Gbest = []
    best_fitness = 0.0
    for i in range(self.populationNumber):
      for j in range(self.dim):
        Xarray[i][j] = (self.Xmax[j] - self.Xmin[j]) * self.rng.uniform(0,1) + self.Xmin[j]
        XbestArray[i][j] = Xarray[i][j]
      fit = self.Fitness(Xarray[i], self.timeScale)
      if len(Gbest) == 0 or self.Fitness(Gbest, self.timeScale) < fit:
        Gbest = Xarray[i].copy()
        best_fitness = fit
      for j in range(self.dim):
        d = self.Xmax[j] - self.Xmin[j]
        Varray[i][j] = self.rng.uniform(-d , d)

    for k in range(self.max_iter):
      for i in range(self.populationNumber):
        rp = self.rng.uniform(0, 1, size=(self.dim))
        rg = self.rng.uniform(0, 1, size=(self.dim))
        Varray[i] = self.w * Varray[i] + self.sigma_p * rp * (XbestArray[i] - Xarray[i])
        + self.sigma_g * rg * (Gbest - Xarray[i])

        Xarray[i] = Xarray[i] + Varray[i]

        Xarray[i] = np.clip(Xarray[i], self.Xmin, self.Xmax)


        fit = self.Fitness(Xarray[i], self.timeScale)
        if fit > self.Fitness(XbestArray[i], self.timeScale):
          XbestArray[i] = Xarray[i]
          if fit > self.Fitness(Gbest, self.timeScale):
            Gbest = Xarray[i].copy()
            best_fitness = fit
      print("fitness:", best_fitness)

    return Gbest, best_fitness


### 3.3 Simple Moving‑Average trading evaluator

In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

class SMA_Evaluator:
    def __init__(self):
        self.Reset()
    
    def Reset(self):
        #self.close_data = []
        #self.open_data = []

        self.sma_diff = []
        self.sma_short = []
        self.sma_long = []

        self.numBuy = 0
        self.numSell = 0

        self.bitcoin = 0
        self.usd = 1000.0
        self.one_minus_fee_rate = 0.97
        return 
    
    def Read_test_data(self, path):
        df = pd.read_csv(path)
        self.close_data = np.array(df['close'])
        self.open_data = np.array(df['open'])

    def Fitness(self , X, scale):
        self.Reset()
        x0 = int(np.round(X[0]))
        x1 = int(np.round(X[1]))
        #x2 = int(np.round(X[2])) #time_scale
        x2 = scale

        averageUsd = 0
        numLoop = len(self.close_data)//x2

        self.sma_short = self.wma(self.close_data, x0, self.sma_filter(x0))
        self.sma_long = self.wma(self.close_data, x1, self.sma_filter(x1))
        self.sma_diff = self.sma_short - self.sma_long
        sign_data = np.sign(self.sma_diff)
        self.signal = self.wma(sign_data, 2, [0.5, -0.5])
        for k in range(numLoop):
          usd = 1000.0
          bitcoin = 0.0
          close_data = self.close_data[k * x2 : k * x2 + x2]
          sma_short = self.wma(close_data, x0, self.sma_filter(x0))
          sma_long =  self.wma(close_data, x1, self.sma_filter(x1))
          sign_data = np.sign(sma_short - sma_long)
          result  = self.wma(sign_data, 2, [0.5, -0.5])
          for i in range(1, len(result)):
              d = close_data[i]
              #buy
              if result[i] > 0.5:
                  bitcoin = bitcoin + usd * self.one_minus_fee_rate / d
                  usd = 0.0
                  self.numBuy = self.numBuy + 1.0
              #sell
              elif result[i] < -0.5:
                  usd = usd + bitcoin * d * self.one_minus_fee_rate
                  bitcoin = 0.0
                  self.numSell = self.numSell + 1.0
        
          if bitcoin > 0:
              usd = usd +  bitcoin * close_data[-1] * self.one_minus_fee_rate
        
          averageUsd += usd
        averageUsd = averageUsd/numLoop
        return averageUsd
    
    def pad(self, P,N):
        #padding = -np.flip(P[1:N])
        padding = np.ones(N - 1) * P[0]
        return  np.append(padding, P)
    
    def sma_filter(self, N):
        return np.ones(N)/N
        
    def wma(self, P,N,kernel):
        return np.convolve(self.pad(P,N), kernel, 'valid')
        







## 4. SMA trading bot tuning

the code from **`train_woa_sma.py`**.  


In [ ]:
from SMA_bot import *
from woa import *
from pso import *
from gwo import *
import numpy as np
import time

def draw(data , sma_short, sma_long, sma_diff):
    x = np.arange(len(data))
    plt.plot(x, data, color = 'black')
    plt.plot(x, sma_short, color='blue')
    plt.plot(x, sma_long, color ='red')
    plt.plot(x, sma_diff, color ='gray' , linestyle='--')

    plt.title("Float Array Plot")
    plt.xlabel("Index")
    plt.ylabel("Value")

    plt.show()

if __name__ == "__main__":
    bot = SMA_Evaluator()
    #bot.Read_test_data("btc_hourly.csv")
    bot.Read_test_data("BTC-Daily.csv") 
    cof = 1
    timeScale = 200
    #bot.close_data = [200 - i for i in range(100)] + [ i + 100 for i in range(200)] + [ 300 - i for i in range(100)]
    
    usd = bot.Fitness(np.array([10 * cof, 20 * cof]), timeScale)

    print("\n")
    print("final usd:", usd)
    print("num buy:" , bot.numBuy)
    print("num sell:", bot.numSell)

    #draw(bot.close_data, bot.sma_short, bot.sma_long, bot.signal * 100)

    woa = WOA_Optimizer(bot.Fitness, 42, 2, [5, 5]*cof, [25, 25]*cof, 500, 100, timeScale)
    X_target = woa.Run()
    fit = woa.Fitness(X_target, timeScale)
    print("best_result:", X_target)
    print("fitness:" , fit)

    #test Pso
    pso = PSO(bot.Fitness, int(time.time()), 2, [5, 5]*cof, [25, 25]*cof, 500, 100, timeScale)
    pso_target = pso.Run()
    pso_fit = pso.Fitness(pso_target, timeScale)
    print("pso_best_result:", pso_target)
    print("pso_fitness:" , pso_fit)

    #test Grey Wolf Optimizer
    gwo = GreyWolfOptimiser(
        fitness_function=bot.Fitness,
        max_iter=500,
        num_wolves=100,
        dim=2,
        minx=[5, 5],
        maxx=[25, 25],
        seed=42,
        fitness_args=(timeScale,)
    )
    gwo_target, gwo_fit = gwo.run()
    print("gwo_best_result:", gwo_target)
    print("gwo_fitness:", gwo_fit)

    print("end")


## 5. Quick benchmark – sphere function

the experiment from **`test_woa.py`**.  
The fitness function is \(-\sum x_i^2\) 


In [ ]:
from woa import *

# sphere function
def fitness_sphere(X):
    fitness_value = 0.0
    for i in range(len(X)):
        xi = X[i]
        fitness_value -= (xi * xi)
    return fitness_value


if __name__ == "__main__":
    dim = 4
    Xmin = [0] * dim
    Xmax = [10] * dim
    woa = WOA_Optimizer(fitness_sphere, 42, 4, Xmin, Xmax, 200, 500)

    X_target = woa.Run()
    fit = woa.Fitness(X_target)
    print("best_result:", X_target)
    print("fitness:" , fit)